# 🚀 HVAC AI — Inference Server (Final, v2)

This notebook provides a robust, one-click solution for deploying the HVAC AI backend on a Google Colab GPU. It clones the repository, installs dependencies, and correctly launches the FastAPI server with a public `ngrok` URL.

**Instructions:**
1. Run the cells in order.
2. When you reach **Cell 4**, edit it to add your `NGROK_AUTHTOKEN` and verify the `MODEL_PATH`.
3. Run the remaining cells to launch the server.

### Phase 1: Environment Setup

In [ ]:
# Cell 1 — Clone repo and change to project root
!git clone https://github.com/elliotttmiller/hvac-ai.git || true
%cd hvac-ai
print('✅ Repo ready at /content/hvac-ai')

In [ ]:
# Cell 2 — Install dependencies
!pip install -q fastapi uvicorn python-multipart flask flask-cors pyngrok python-dotenv nest-asyncio
!pip install -q torch torchvision --extra-index-url https://download.pytorch.org/whl/cu118 || true
!pip install -q opencv-python pycocotools matplotlib onnxruntime onnx || true
!pip install -q git+https://github.com/facebookresearch/segment-anything.git || true
print('✅ Dependencies installation attempted.')

In [ ]:
# Cell 3 — Mount Google Drive
from google.colab import drive
print('Mounting Google Drive to access the fine-tuned model...')
drive.mount('/content/drive')
print('✅ Drive mounted.')

In [ ]:
%%writefile .env
# ⚠️ ACTION REQUIRED: Edit these values before running the launch cell

# 1. Get your token from https://dashboard.ngrok.com/get-started/your-authtoken and paste it here
NGROK_AUTHTOKEN="REPLACE_WITH_YOUR_NGROK_AUTHTOKEN"

# 2. Verify this path points to your fine-tuned model on Google Drive
MODEL_PATH="/content/drive/MyDrive/sam_finetuning_results/latest_checkpoint_multiprompt_v1.pth"

# 3. The port for the server. This must match the port in the launch cell.
PORT=8000
print('✅ .env file created. Please edit the placeholder values before proceeding.')

In [ ]:
# Cell 5 — Authenticate ngrok (reads from .env)
import os
from dotenv import load_dotenv
load_dotenv()
ngrok_token = os.getenv('NGROK_AUTHTOKEN')
if not ngrok_token or 'REPLACE_WITH' in ngrok_token:
 raise SystemExit('❌ CRITICAL: NGROK_AUTHTOKEN is missing or is still a placeholder. Please edit the .env cell above and rerun it.')
print('Authenticating ngrok...')
!ngrok authtoken {ngrok_token}
print('✅ ngrok authtoken set successfully.')

In [ ]:
# Cell 6 — Launch the Server
import os
from pyngrok import ngrok
from dotenv import load_dotenv
load_dotenv()

PORT = int(os.getenv('PORT', '8000'))

# Ensure any stale tunnels are closed for a clean start
ngrok.kill()

# Start the ngrok tunnel
public_url = ngrok.connect(PORT).public_url
print('-' * 60)
print(f'✅ Your inference server is live at: {public_url}')
print(f'👉 Use this URL for the NEXT_PUBLIC_AI_SERVICE_URL in your frontend .env.local file.')
print('-' * 60)

print('\n🚀 Starting FastAPI backend... This cell will run continuously and show live server logs.')

# Change directory into the services folder so the module import is valid
%cd python-services

# Run uvicorn using the correct module:app syntax. This blocks the cell.
!uvicorn hvac_analysis_service:app --host 0.0.0.0 --port {PORT} --reload

### Notes & Troubleshooting
- **`FileNotFoundError` on startup?** The most likely cause is an incorrect `MODEL_PATH` in your `.env` file. Double-check the path in your Google Drive.
- **`ERR_NGROK_8012` (Connection Refused)?** This notebook is designed to prevent this by using the same `PORT` variable for both `ngrok` and `uvicorn`. If it still occurs, it means the `uvicorn` server crashed. Check the cell output for Python errors.
- **Frontend Connection:** After the server starts and you see the public URL, update your frontend's `.env.local` file, then **restart your Next.js dev server** (`npm run dev`) for the change to take effect.